In [2]:
# Import necessary libraries
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from sklearn.tree import plot_tree
from sklearn.model_selection import train_test_split
from sksurv.ensemble import RandomSurvivalForest
from sksurv.linear_model import CoxPHSurvivalAnalysis
from sksurv.metrics import concordance_index_censored , concordance_index_ipcw
from sklearn.impute import SimpleImputer
from sksurv.util import Surv

# Clinical Data
df = pd.read_csv("data/X_train/clinical_train.csv")
df_eval = pd.read_csv("data/X_test/clinical_test.csv")

# Molecular Data
maf_df = pd.read_csv("data/X_train/molecular_train.csv")
maf_eval = pd.read_csv("data/X_test/molecular_test.csv")

target_df = pd.read_csv("data/target_train.csv")
target_df_test = pd.read_csv("data/random_submission_FRacdcw_v9kP4pP.csv")

# Preview the data
df.head()

,ID,CENTER,BM_BLAST,WBC,ANC,MONOCYTES,HB,PLT,CYTOGENETICS
0,P132697,MSK,14.0,2.8,0.2,0.7,7.6,119.0,"46,xy,del(20)(q12)[2]/46,xy[18]"
1,P132698,MSK,1.0,7.4,2.4,0.1,11.6,42.0,"46,xx"
2,P116889,MSK,15.0,3.7,2.1,0.1,14.2,81.0,"46,xy,t(3;3)(q25;q27)[8]/46,xy[12]"
3,P132699,MSK,1.0,3.9,1.9,0.1,8.9,77.0,"46,xy,del(3)(q26q27)[15]/46,xy[5]"
4,P132700,MSK,6.0,128.0,9.7,0.9,11.1,195.0,"46,xx,t(3;9)(p13;q22)[10]/46,xx[10]"


In [3]:
def process_y(target_df):
    # Drop rows where 'OS_YEARS' is NaN if conversion caused any issues
    target_df.dropna(subset=['OS_YEARS', 'OS_STATUS'], inplace=True)

    # Contarget_dfvert 'OS_YEARS' to numeric if it isn’t already
    target_df['OS_YEARS'] = pd.to_numeric(target_df['OS_YEARS'], errors='coerce')

    # Ensure 'OS_STATUS' is boolean
    target_df['OS_STATUS'] = target_df['OS_STATUS'].astype(bool)

    return Surv.from_dataframe('OS_STATUS', 'OS_YEARS', target_df)

import featuretools as ft

def create_entity(status):

    clinical = pd.read_csv(f"data/X_{status}/clinical_{status}.csv")
    molecular = pd.read_csv(f"data/X_{status}/molecular_{status}.csv").reset_index()
    
    es = ft.EntitySet(id=status)

    es = es.add_dataframe(
        dataframe_name='clinical',
        dataframe=clinical,
        index='ID' 
    )

    es = es.add_dataframe(
        dataframe_name='molecular',
        dataframe=molecular,
        index='index'   
    )

    es.add_relationship(
        parent_dataframe_name='clinical',
        parent_column_name='ID',
        child_dataframe_name='molecular',
        child_column_name='ID'
    )

    return es

2025-01-09 16:48:37,621 featuretools - WARNING    While loading primitives via "premium_primitives" entry point, ignored primitive "DiversityScore" from "premium_primitives.diversity_score" because a primitive with that name already exists in "nlp_primitives.diversity_score"
2025-01-09 16:48:37,622 featuretools - WARNING    While loading primitives via "premium_primitives" entry point, ignored primitive "LSA" from "premium_primitives.lsa" because a primitive with that name already exists in "nlp_primitives.lsa"
2025-01-09 16:48:37,622 featuretools - WARNING    While loading primitives via "premium_primitives" entry point, ignored primitive "MeanCharactersPerSentence" from "premium_primitives.mean_characters_per_sentence" because a primitive with that name already exists in "nlp_primitives.mean_characters_per_sentence"
2025-01-09 16:48:37,622 featuretools - WARNING    While loading primitives via "premium_primitives" entry point, ignored primitive "NumberOfSentences" from "premium_primi

In [ ]:
train = create_entity('train')
X, features_defs = ft.dfs(entityset=train, target_dataframe_name="clinical")

target = load_target()

# Create the survival data format
X = X.loc[X.index.isin(target['ID'])]
y = Surv.from_dataframe('OS_STATUS', 'OS_YEARS', target)

# Sélectionner les colonnes catégorielles
categorical_columns = X.select_dtypes(include=['category']).columns

# Retirer les colonnes catégorielles
X = X.drop(categorical_columns, axis=1)




/Users/julesmourgues/Documents/Programmation/Personal/qrt-challenge-2025/.venv/lib/python3.11/site-packages/woodwork/type_sys/utils.py:33: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(
/Users/julesmourgues/Documents/Programmation/Personal/qrt-challenge-2025/.venv/lib/python3.11/site-packages/woodwork/type_sys/utils.py:33: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(
/Users/julesmourgues/Documents/Programmation/Personal/qrt-challenge-2025/.venv/lib/python3.11/site-packages/woodwork/type_sys/utils.py:33: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a for

Index(['CENTER', 'BM_BLAST', 'WBC', 'ANC', 'MONOCYTES', 'HB', 'PLT',
       'COUNT(molecular)', 'MAX(molecular.DEPTH)', 'MAX(molecular.END)',
       'MAX(molecular.START)', 'MAX(molecular.VAF)', 'MEAN(molecular.DEPTH)',
       'MEAN(molecular.END)', 'MEAN(molecular.START)', 'MEAN(molecular.VAF)',
       'MIN(molecular.DEPTH)', 'MIN(molecular.END)', 'MIN(molecular.START)',
       'MIN(molecular.VAF)', 'MODE(molecular.ALT)', 'MODE(molecular.CHR)',
       'MODE(molecular.EFFECT)', 'MODE(molecular.GENE)', 'MODE(molecular.REF)',
       'NUM_UNIQUE(molecular.ALT)', 'NUM_UNIQUE(molecular.CHR)',
       'NUM_UNIQUE(molecular.EFFECT)', 'NUM_UNIQUE(molecular.GENE)',
       'NUM_UNIQUE(molecular.REF)', 'SKEW(molecular.DEPTH)',
       'SKEW(molecular.END)', 'SKEW(molecular.START)', 'SKEW(molecular.VAF)',
       'STD(molecular.DEPTH)', 'STD(molecular.END)', 'STD(molecular.START)',
       'STD(molecular.VAF)', 'SUM(molecular.DEPTH)', 'SUM(molecular.END)',
       'SUM(molecular.START)', 'SUM(molecular

/Users/julesmourgues/Documents/Programmation/Personal/qrt-challenge-2025/.venv/lib/python3.11/site-packages/featuretools/computational_backends/feature_set_calculator.py:785: FutureWarning: The provided callable <function sum at 0x1072e37e0> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  ).agg(to_agg)
/Users/julesmourgues/Documents/Programmation/Personal/qrt-challenge-2025/.venv/lib/python3.11/site-packages/featuretools/computational_backends/feature_set_calculator.py:785: FutureWarning: The provided callable <function std at 0x107310d60> is currently using SeriesGroupBy.std. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "std" instead.
  ).agg(to_agg)
/Users/julesmourgues/Documents/Programmation/Personal/qrt-challenge-2025/.venv/lib/python3.11/site-packages/featuretools/computational_backends/

In [5]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


In [18]:
# Import necessary libraries
import lightgbm as lgb
from sksurv.metrics import concordance_index_censored
from sksurv.util import Surv

# Define LightGBM parameters
lgbm_params = {
    'max_depth': 2,
    'learning_rate': 0.05,
    'verbose': -1
}

# Prepare the data for LightGBM
# Scale the target (OS_YEARS) to reduce skew, apply weights based on event status
X_train_lgb = X_train  # Features for training
y_train_transformed = y_train['OS_YEARS']

# Create LightGBM dataset
train_dataset = lgb.Dataset(X_train_lgb, label=y_train_transformed)

# Train the LightGBM model
model = lgb.train(params=lgbm_params, train_set=train_dataset)

# Make predictions on the training and testing sets
pred_train = -model.predict(X_train)
pred_test = -model.predict(X_test)

# Evaluate the model using Concordance Index IPCW
train_ci_ipcw = concordance_index_ipcw(y_train, y_train, pred_train, tau=7)[0]
test_ci_ipcw = concordance_index_ipcw(y_train, y_test, pred_test, tau=7)[0]
print(f"LightGBM Survival Model Concordance Index IPCW on train: {train_ci_ipcw:.2f}")
print(f"LightGBM Survival Model Concordance Index IPCW on test: {test_ci_ipcw:.2f}")


LightGBM Survival Model Concordance Index IPCW on train: 0.73
LightGBM Survival Model Concordance Index IPCW on test: 0.69


In [7]:
# Survival-aware imputation for missing values
imputer = SimpleImputer(strategy="median")
X_train = imputer.fit_transform(X_train)
X_test = imputer.transform(X_test)

In [8]:
# Initialize and train the Cox Proportional Hazards model
cox = CoxPHSurvivalAnalysis()
cox.fit(X_train, y_train)

# Evaluate the model using Concordance Index IPCW
cox_cindex_train = concordance_index_ipcw(y_train, y_train, cox.predict(X_train), tau=7)[0]
cox_cindex_test = concordance_index_ipcw(y_train, y_test, cox.predict(X_test), tau=7)[0]
print(f"Cox Proportional Hazard Model Concordance Index IPCW on train: {cox_cindex_train:.2f}")
print(f"Cox Proportional Hazard Model Concordance Index IPCW on test: {cox_cindex_test:.2f}")

/Users/julesmourgues/Documents/Programmation/Personal/qrt-challenge-2025/.venv/lib/python3.11/site-packages/sksurv/linear_model/coxph.py:449: LinAlgWarning: Ill-conditioned matrix (rcond=3.24041e-29): result may not be accurate.
  delta = solve(
/Users/julesmourgues/Documents/Programmation/Personal/qrt-challenge-2025/.venv/lib/python3.11/site-packages/sksurv/linear_model/coxph.py:449: LinAlgWarning: Ill-conditioned matrix (rcond=1.78327e-29): result may not be accurate.
  delta = solve(
/Users/julesmourgues/Documents/Programmation/Personal/qrt-challenge-2025/.venv/lib/python3.11/site-packages/sksurv/linear_model/coxph.py:449: LinAlgWarning: Ill-conditioned matrix (rcond=1.76974e-29): result may not be accurate.
  delta = solve(
/Users/julesmourgues/Documents/Programmation/Personal/qrt-challenge-2025/.venv/lib/python3.11/site-packages/sksurv/linear_model/coxph.py:449: LinAlgWarning: Ill-conditioned matrix (rcond=1.02554e-29): result may not be accurate.
  delta = solve(
/Users/julesmour

Cox Proportional Hazard Model Concordance Index IPCW on train: 0.70
Cox Proportional Hazard Model Concordance Index IPCW on test: 0.67


/Users/julesmourgues/Documents/Programmation/Personal/qrt-challenge-2025/.venv/lib/python3.11/site-packages/sksurv/linear_model/coxph.py:449: LinAlgWarning: Ill-conditioned matrix (rcond=1.77163e-29): result may not be accurate.
  delta = solve(
/Users/julesmourgues/Documents/Programmation/Personal/qrt-challenge-2025/.venv/lib/python3.11/site-packages/sksurv/linear_model/coxph.py:449: LinAlgWarning: Ill-conditioned matrix (rcond=3.86392e-29): result may not be accurate.
  delta = solve(
/var/folders/vk/z0p4r5z97d99dp7mpbt9yd1w0000gn/T/ipykernel_6769/1910974199.py:3: ConvergenceWarning: Optimization did not converge: Maximum number of iterations has been exceeded.
  cox.fit(X_train, y_train)


# Predict

In [9]:
test = create_entity('test')
X_eval, features_defs = ft.dfs(entityset=test, target_dataframe_name="clinical")

# Sélectionner les colonnes catégorielles
categorical_columns = X_eval.select_dtypes(include=['category']).columns

# Retirer les colonnes catégorielles
X_eval = X_eval.drop(categorical_columns, axis=1)

# Survival-aware imputation for missing values
X_eval = imputer.transform(X_eval)



/Users/julesmourgues/Documents/Programmation/Personal/qrt-challenge-2025/.venv/lib/python3.11/site-packages/woodwork/type_sys/utils.py:33: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(
/Users/julesmourgues/Documents/Programmation/Personal/qrt-challenge-2025/.venv/lib/python3.11/site-packages/woodwork/type_sys/utils.py:33: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(
/Users/julesmourgues/Documents/Programmation/Personal/qrt-challenge-2025/.venv/lib/python3.11/site-packages/woodwork/type_sys/utils.py:33: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a for